In [1]:
from ml_core import ML_core
import pandas as pd

In [9]:
train_data = pd.read_csv('data/name_attached/train_data_incl_names.csv')
predict_data = pd.read_csv('data/name_attached/prediction_data_incl_names.csv') 


In [11]:
ml_core = ML_core(train_data, predict_data)
ml_core.train()
result = ml_core.predict()

In [14]:
r.to 

,Unnamed: 0,Name,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,999,Mi Richan,42,Travel_Rarely,1147,Human Resources,10,3,Human Resources,1,...,80,1,21,5,3,20,7,0,9,0
1,1000,Francoise Rautenstrauch,52,Travel_Rarely,258,Research & Development,8,4,Other,1,...,80,0,12,2,1,5,4,0,4,0
2,1001,Kendra Loud,37,Travel_Rarely,1462,Research & Development,11,3,Medical,1,...,80,0,8,6,3,3,2,0,2,0
3,1002,Lourdes Bauswell,35,Travel_Frequently,200,Research & Development,18,2,Life Sciences,1,...,80,0,10,2,3,2,2,2,2,0
4,1003,Hannah Edmison,25,Travel_Rarely,949,Research & Development,1,3,Technical Degree,1,...,80,1,7,2,2,3,2,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,1465,Rene Granier,36,Travel_Frequently,884,Research & Development,23,2,Medical,1,...,80,1,17,3,3,5,2,0,3,0
467,1466,Trevor Harnist,39,Travel_Rarely,613,Research & Development,6,1,Medical,1,...,80,1,9,5,3,7,7,1,7,0
468,1467,Sharmaine Batdorf,27,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,...,80,1,6,0,3,6,2,0,3,0
469,1468,Ricki Traux,49,Travel_Frequently,1023,Sales,2,3,Medical,1,...,80,0,17,3,2,9,6,0,8,0


In [ ]:
r.to_csv()